# Astro Vibe Bot — Local Test Notebook

Use this notebook to generate a daily context, preview the channel message for all 12 signs, and optionally send it to a Telegram chat or channel.

Prereqs:
- `.env` must exist with `TELEGRAM_BOT_TOKEN` and `OPENAI_API_KEY`
- Optional: set `RSS_FEED_URL`, `TIMEZONE`, and `TEST_CHAT_ID` in `.env`


In [1]:
import os
import asyncio
from pathlib import Path
from zoneinfo import ZoneInfo

from dotenv import load_dotenv
from openai import OpenAI
from aiogram import Bot

from main import load_signs, generate_daily_context, build_all_signs_message

root_dir = Path.cwd()
if not (root_dir / ".env").exists() and (root_dir.parent / ".env").exists():
    root_dir = root_dir.parent
load_dotenv(dotenv_path=root_dir / ".env")

TELEGRAM_BOT_TOKEN = os.getenv("TELEGRAM_BOT_TOKEN")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
RSS_FEED_URL = os.getenv("RSS_FEED_URL")
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-4o-mini")
TEST_CHAT_ID = os.getenv("TEST_CHAT_ID")
TIMEZONE = os.getenv("TIMEZONE", "UTC")
TELEGRAM_API_ID = os.getenv("TELEGRAM_API_ID")
TELEGRAM_API_HASH = os.getenv("TELEGRAM_API_HASH")
TELEGRAM_NEWS_CHANNEL = os.getenv("TELEGRAM_NEWS_CHANNEL")
TELEGRAM_NEWS_LIMIT = int(os.getenv("TELEGRAM_NEWS_LIMIT", "20"))
TELETHON_SESSION = os.getenv("TELETHON_SESSION", "telethon.session")

if not TELEGRAM_BOT_TOKEN or not OPENAI_API_KEY:
    raise RuntimeError("Set TELEGRAM_BOT_TOKEN and OPENAI_API_KEY in .env")

client = OpenAI(api_key=OPENAI_API_KEY)
signs = load_signs()
timezone = ZoneInfo(TIMEZONE)
telegram_source = {
    "api_id": int(TELEGRAM_API_ID) if TELEGRAM_API_ID else None,
    "api_hash": TELEGRAM_API_HASH,
    "channel": TELEGRAM_NEWS_CHANNEL,
    "limit": TELEGRAM_NEWS_LIMIT,
    "session_path": TELETHON_SESSION,
}


In [2]:
# Generate today's global context
context = await generate_daily_context(
    client,
    signs,
    RSS_FEED_URL,
    OPENAI_MODEL,
    telegram_source=telegram_source,
)
context["global_summary"], list(context["vibes"].keys())[:3]


Starting to build telegram news context
[debug] Telegram news context:
Немає налаштованого джерела новин.
[/debug]


('Сьогоднішні новини принесли багато викликів та тривог. Проте, кожен знак зодіаку має можливість знайти сили в собі, щоб подолати ці труднощі та рухатися вперед.',
 ['Aries', 'Taurus', 'Gemini'])

In [3]:
# Build the daily channel message (all 12 signs)
channel_message = build_all_signs_message(context, signs, timezone)
channel_message


'Вайб дня — 2026-02-07\n\nОвен: Сьогодні імпульс та енергія відкрити нові горизонти! Використай свою сміливість, щоб втілити ідеї в життя.\n\nТелець: Стабільність приносить спокій, а терпіння відкриває нові можливості. Сьогодні ідеальний день для планування майбутнього.\n\nБлизнюки: Твоє допитливе серце штовхає вперед! Використай цю гнучкість, щоб дослідити нові ідеї та обговорити їх з друзями.\n\nРак: Твоя чутливість сьогодні може стати силою. Проводь час з близькими, адже турбота про них дарує тобі натхнення.\n\nЛев: Твоя впевненість сьогодні сяє, як ніколи! Використай свій лідерський потенціал, щоб надихнути інших.\n\nДіва: Аналізуй та плануй з точністю! Сьогодні ідеальний день, щоб впорядкувати справи та внести ясність у свої цілі.\n\nТерези: Шукай гармонію в стосунках і насолоджуйся спілкуванням. Твоя дипломатичність допоможе вирішити будь-які питання.\n\nСкорпіон: Твоя інтенсивність може допомогти заглибитися в важливі справи. Довіряй своїй інтуїції, вона веде до успіху.\n\nСтріл

In [4]:
# Optional: send the daily channel message
# Set TEST_CHAT_ID in .env (example: your numeric chat_id or @TarotReaderOracle)

if TEST_CHAT_ID:
    bot = Bot(token=TELEGRAM_BOT_TOKEN)
    await bot.send_message(TEST_CHAT_ID, channel_message)
    await bot.session.close()
    "Sent"
else:
    "TEST_CHAT_ID not set"
